In [2]:
import numpy as np
import pandas as pd
# Carga los datos del archivo CSV utilizando Pandas.
ruta_archivo = '../data/ejercicios_integracion/ventas_tienda_simulada.csv'
datos_pd = pd.read_csv(ruta_archivo, sep=',',header=0)

datos_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fecha            20 non-null     object 
 1   producto         20 non-null     object 
 2   cantidad         16 non-null     float64
 3   precio_unitario  16 non-null     float64
 4   total            12 non-null     float64
dtypes: float64(3), object(2)
memory usage: 932.0+ bytes


In [3]:
# Vemos si tiene datos nulos.
datos_pd.isnull().sum()

fecha              0
producto           0
cantidad           4
precio_unitario    4
total              8
dtype: int64

In [4]:
# Ver los registros nulos
datos_pd[datos_pd.isnull().any(axis=1)]


,fecha,producto,cantidad,precio_unitario,total
1,2023-01-02,Producto_B,NaN,20.0,NaN
2,2023-01-03,Producto_C,5.0,NaN,NaN
6,2023-01-07,Producto_C,NaN,NaN,NaN
8,2023-01-09,Producto_A,2.0,10.0,NaN
9,2023-01-10,Producto_B,NaN,NaN,NaN
13,2023-01-14,Producto_B,NaN,25.0,NaN
16,2023-01-17,Producto_A,8.0,10.0,NaN
18,2023-01-19,Producto_C,7.0,NaN,NaN


In [9]:
# Calcular el total de los que tienen cantidad y precio unitario pero el total en nulo.

# Filtra los registros donde 'precio_unitario' y 'cantidad' no son nulos y 'total' es nulo
condicion = datos_pd['precio_unitario'].notnull() & datos_pd['cantidad'].notnull() & datos_pd['total'].isnull()

# Calcula el total y actualiza la columna 'total'
datos_pd.loc[condicion, 'total'] = datos_pd.loc[condicion, 'cantidad'] * datos_pd.loc[condicion, 'precio_unitario']

# Muestra el DataFrame actualizado
print(datos_pd)


         fecha    producto  cantidad  precio_unitario  total
0   2023-01-01  Producto_A       2.0             10.5   21.0
1   2023-01-02  Producto_B       NaN             20.0    NaN
2   2023-01-03  Producto_C       5.0              NaN    NaN
3   2023-01-04  Producto_D       7.0             15.0  105.0
4   2023-01-05  Producto_A       1.0             12.0   12.0
5   2023-01-06  Producto_B       4.0             25.0  100.0
6   2023-01-07  Producto_C       NaN              NaN    NaN
7   2023-01-08  Producto_D       3.0             30.0   90.0
8   2023-01-09  Producto_A       2.0             10.0   20.0
9   2023-01-10  Producto_B       NaN              NaN    NaN
10  2023-01-11  Producto_C       6.0             22.0  132.0
11  2023-01-12  Producto_D       4.0             18.0   72.0
12  2023-01-13  Producto_A       5.0             20.0  100.0
13  2023-01-14  Producto_B       NaN             25.0    NaN
14  2023-01-15  Producto_C       1.0             14.0   14.0
15  2023-01-16  Producto

In [10]:
# Ver los registros nulos
datos_pd[datos_pd.isnull().any(axis=1)]

,fecha,producto,cantidad,precio_unitario,total
1,2023-01-02,Producto_B,NaN,20.0,NaN
2,2023-01-03,Producto_C,5.0,NaN,NaN
6,2023-01-07,Producto_C,NaN,NaN,NaN
9,2023-01-10,Producto_B,NaN,NaN,NaN
13,2023-01-14,Producto_B,NaN,25.0,NaN
18,2023-01-19,Producto_C,7.0,NaN,NaN


In [11]:
# Ver los precios
productos_precios_unicos = datos_pd[['producto', 'precio_unitario']].drop_duplicates()
productos_precios_unicos

,producto,precio_unitario
0,Producto_A,10.5
1,Producto_B,20.0
2,Producto_C,NaN
3,Producto_D,15.0
4,Producto_A,12.0
5,Producto_B,25.0
7,Producto_D,30.0
8,Producto_A,10.0
9,Producto_B,NaN
10,Producto_C,22.0


In [17]:
# Filtrar el DataFrame para obtener solo los registros del producto específico
precios_producto_b = datos_pd[datos_pd['producto'] == 'Producto_B']['precio_unitario']
#precios_producto_b
precios_producto_b.describe()

count     4.000000
mean     20.250000
std       6.601767
min      11.000000
25%      17.750000
50%      22.500000
75%      25.000000
max      25.000000
Name: precio_unitario, dtype: float64

In [18]:
# Asingo el valor del primer quartil
precio_asignar = 17.75

# Filtra los registros donde el producto es 'ProductoB' y el precio es nulo
condicion = (datos_pd['producto'] == 'Producto_B') & (datos_pd['precio_unitario'].isnull())

# Asigna el precio al producto donde es nulo
datos_pd.loc[condicion, 'precio_unitario'] = precio_asignar

# Muestra el DataFrame actualizado
print(datos_pd)


         fecha    producto  cantidad  precio_unitario  total
0   2023-01-01  Producto_A       2.0            10.50   21.0
1   2023-01-02  Producto_B       NaN            20.00    NaN
2   2023-01-03  Producto_C       5.0              NaN    NaN
3   2023-01-04  Producto_D       7.0            15.00  105.0
4   2023-01-05  Producto_A       1.0            12.00   12.0
5   2023-01-06  Producto_B       4.0            25.00  100.0
6   2023-01-07  Producto_C       NaN              NaN    NaN
7   2023-01-08  Producto_D       3.0            30.00   90.0
8   2023-01-09  Producto_A       2.0            10.00   20.0
9   2023-01-10  Producto_B       NaN            17.75    NaN
10  2023-01-11  Producto_C       6.0            22.00  132.0
11  2023-01-12  Producto_D       4.0            18.00   72.0
12  2023-01-13  Producto_A       5.0            20.00  100.0
13  2023-01-14  Producto_B       NaN            25.00    NaN
14  2023-01-15  Producto_C       1.0            14.00   14.0
15  2023-01-16  Producto

In [19]:
# Filtrar el DataFrame para obtener solo los registros del producto específico
cantidad_producto_b = datos_pd[datos_pd['producto'] == 'Producto_B']['cantidad']
#cantidad_producto_b
cantidad_producto_b.describe()

count    2.0
mean     4.0
std      0.0
min      4.0
25%      4.0
50%      4.0
75%      4.0
max      4.0
Name: cantidad, dtype: float64

In [20]:
# Asignamos 4 a cantidad del producto b
# Asingo el valor del primer quartil
cantidad_asignar = 4

# Filtra los registros donde el producto es 'ProductoB' y el precio es nulo
condicion_cantidad = (datos_pd['producto'] == 'Producto_B') & (datos_pd['cantidad'].isnull())

# Asigna el precio al producto donde es nulo
datos_pd.loc[condicion_cantidad, 'cantidad'] = cantidad_asignar

# Muestra el DataFrame actualizado
print(datos_pd)

         fecha    producto  cantidad  precio_unitario  total
0   2023-01-01  Producto_A       2.0            10.50   21.0
1   2023-01-02  Producto_B       4.0            20.00    NaN
2   2023-01-03  Producto_C       5.0              NaN    NaN
3   2023-01-04  Producto_D       7.0            15.00  105.0
4   2023-01-05  Producto_A       1.0            12.00   12.0
5   2023-01-06  Producto_B       4.0            25.00  100.0
6   2023-01-07  Producto_C       NaN              NaN    NaN
7   2023-01-08  Producto_D       3.0            30.00   90.0
8   2023-01-09  Producto_A       2.0            10.00   20.0
9   2023-01-10  Producto_B       4.0            17.75    NaN
10  2023-01-11  Producto_C       6.0            22.00  132.0
11  2023-01-12  Producto_D       4.0            18.00   72.0
12  2023-01-13  Producto_A       5.0            20.00  100.0
13  2023-01-14  Producto_B       4.0            25.00    NaN
14  2023-01-15  Producto_C       1.0            14.00   14.0
15  2023-01-16  Producto

In [21]:
# Filtrar el DataFrame para obtener solo los registros del producto específico
precios_producto_c = datos_pd[datos_pd['producto'] == 'Producto_C']['precio_unitario']
#precios_producto_b
precios_producto_c.describe()

count     2.000000
mean     18.000000
std       5.656854
min      14.000000
25%      16.000000
50%      18.000000
75%      20.000000
max      22.000000
Name: precio_unitario, dtype: float64

In [22]:
# Asingo el valor de la mediana
precio_asignar = 18

# Filtra los registros donde el producto es 'ProductoC' y el precio es nulo
condicion = (datos_pd['producto'] == 'Producto_C') & (datos_pd['precio_unitario'].isnull())

# Asigna el precio al producto donde es nulo
datos_pd.loc[condicion, 'precio_unitario'] = precio_asignar

# Muestra el DataFrame actualizado
print(datos_pd)

         fecha    producto  cantidad  precio_unitario  total
0   2023-01-01  Producto_A       2.0            10.50   21.0
1   2023-01-02  Producto_B       4.0            20.00    NaN
2   2023-01-03  Producto_C       5.0            18.00    NaN
3   2023-01-04  Producto_D       7.0            15.00  105.0
4   2023-01-05  Producto_A       1.0            12.00   12.0
5   2023-01-06  Producto_B       4.0            25.00  100.0
6   2023-01-07  Producto_C       NaN            18.00    NaN
7   2023-01-08  Producto_D       3.0            30.00   90.0
8   2023-01-09  Producto_A       2.0            10.00   20.0
9   2023-01-10  Producto_B       4.0            17.75    NaN
10  2023-01-11  Producto_C       6.0            22.00  132.0
11  2023-01-12  Producto_D       4.0            18.00   72.0
12  2023-01-13  Producto_A       5.0            20.00  100.0
13  2023-01-14  Producto_B       4.0            25.00    NaN
14  2023-01-15  Producto_C       1.0            14.00   14.0
15  2023-01-16  Producto

In [23]:
# Filtrar el DataFrame para obtener solo los registros del producto específico
cantidad_producto_c = datos_pd[datos_pd['producto'] == 'Producto_C']['cantidad']
#cantidad_producto_b
cantidad_producto_c.describe()

count    4.000000
mean     4.750000
std      2.629956
min      1.000000
25%      4.000000
50%      5.500000
75%      6.250000
max      7.000000
Name: cantidad, dtype: float64

In [24]:
# Asignamos 5.5 a cantidad del producto c. La mediana
# Asingo el valor del primer quartil
cantidad_asignar = 5.5

# Filtra los registros donde el producto es 'ProductoB' y el precio es nulo
condicion_cantidad = (datos_pd['producto'] == 'Producto_C') & (datos_pd['cantidad'].isnull())

# Asigna el precio al producto donde es nulo
datos_pd.loc[condicion_cantidad, 'cantidad'] = cantidad_asignar

# Muestra el DataFrame actualizado
print(datos_pd)

         fecha    producto  cantidad  precio_unitario  total
0   2023-01-01  Producto_A       2.0            10.50   21.0
1   2023-01-02  Producto_B       4.0            20.00    NaN
2   2023-01-03  Producto_C       5.0            18.00    NaN
3   2023-01-04  Producto_D       7.0            15.00  105.0
4   2023-01-05  Producto_A       1.0            12.00   12.0
5   2023-01-06  Producto_B       4.0            25.00  100.0
6   2023-01-07  Producto_C       5.5            18.00    NaN
7   2023-01-08  Producto_D       3.0            30.00   90.0
8   2023-01-09  Producto_A       2.0            10.00   20.0
9   2023-01-10  Producto_B       4.0            17.75    NaN
10  2023-01-11  Producto_C       6.0            22.00  132.0
11  2023-01-12  Producto_D       4.0            18.00   72.0
12  2023-01-13  Producto_A       5.0            20.00  100.0
13  2023-01-14  Producto_B       4.0            25.00    NaN
14  2023-01-15  Producto_C       1.0            14.00   14.0
15  2023-01-16  Producto

In [26]:
# Ahora tenemos todos precio y cantidad para calcular el total en donde sea nulo
# Filtra los registros donde 'total' es nulo
condicion = datos_pd['total'].isnull()

# Calcula el total y actualiza la columna 'total' donde la condición es verdadera
datos_pd.loc[condicion, 'total'] = datos_pd.loc[condicion, 'cantidad'] * datos_pd.loc[condicion, 'precio_unitario']
datos_pd

,fecha,producto,cantidad,precio_unitario,total
0,2023-01-01,Producto_A,2.0,10.50,21.0
1,2023-01-02,Producto_B,4.0,20.00,80.0
2,2023-01-03,Producto_C,5.0,18.00,90.0
3,2023-01-04,Producto_D,7.0,15.00,105.0
4,2023-01-05,Producto_A,1.0,12.00,12.0
5,2023-01-06,Producto_B,4.0,25.00,100.0
6,2023-01-07,Producto_C,5.5,18.00,99.0
7,2023-01-08,Producto_D,3.0,30.00,90.0
8,2023-01-09,Producto_A,2.0,10.00,20.0
9,2023-01-10,Producto_B,4.0,17.75,71.0


In [27]:
datos_pd.describe()

,cantidad,precio_unitario,total
count,20.000000,20.000000,20.000000
mean,4.275000,16.912500,73.650000
std,1.956602,5.743002,37.564366
min,1.000000,9.000000,12.000000
25%,3.000000,11.750000,39.750000
50%,4.000000,17.875000,85.000000
75%,5.625000,20.000000,100.000000
max,8.000000,30.000000,132.000000


In [28]:
# Ver los registros nulos
datos_pd[datos_pd.isnull().any(axis=1)]

,fecha,producto,cantidad,precio_unitario,total
